# Churn Analysis

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("user_satisfaction_survey.csv")

In [3]:
data.head()

,ID,Classes_per_week,Happy_with_instructors,Happy_with_class_duration,Happy_with_class_timings,Happy_with_class_size,Happy_with_facilities,Happy_with_price,Churn
0,100,3,Yes,Yes,Yes,Yes,Yes,Yes,No
1,101,4,Yes,No,Yes,No,No,No,No
2,102,3,No,Yes,No,Yes,Yes,Yes,Yes
3,103,2,No,Yes,No,Yes,No,Yes,Yes
4,104,4,Yes,Yes,Yes,Yes,Yes,Yes,No


In [4]:
X = data.drop(['Churn','ID'], axis=1) #Dropping the ID column also - its not needed
y = data['Churn']

In [5]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()



OneHotEncoder will be used to turn the categorical (yes, no) to numerical

In [6]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

Column_transformer will be used to apply the OneHotEncoder to all the columns except Classes_per_week (that's already numerical)

In [7]:
from sklearn.compose import make_column_transformer

column_trans = make_column_transformer((OneHotEncoder(),['Happy_with_instructors', 'Happy_with_class_duration',\
                                                       'Happy_with_class_timings', 'Happy_with_class_size',\
                                                       'Happy_with_facilities', 'Happy_with_price']), \
                                       remainder='passthrough')

Uncomment code below to see the column_transformer applying the OneHotEncoder to data 

In [8]:
# column_trans.fit_transform(X)

Use pipeline to chain steps together. Applies 'column_transform' to data and passes it through decision tree classifier

In [9]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(column_trans, dtree)

Apply 5-fold cross validation to pipeline and getting the mean accuracy 

In [10]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

0.96

Taking a sample from our data to apply it to the model

In [83]:
X_new = X.sample(1)
X_new

,Classes_per_week,Happy_with_instructors,Happy_with_class_duration,Happy_with_class_timings,Happy_with_class_size,Happy_with_facilities,Happy_with_price
10,2,Yes,Yes,Yes,Yes,Yes,Yes


Train our pipleline with the data

In [71]:
pipe.fit(X,y)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categorical_features=None,
                                                                categories=None,
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                n_values=None,
                                                                sparse=True),
                                                  ['Happy_with_instructors'...
                

Use our pipeline to make a prediction

In [82]:
pipe.predict(X_new)

array(['Yes'], dtype=object)

Add some data by hand to predict a 'No'

In [85]:
sample1 = [3,'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No']
sample1 = pd.DataFrame([sample1], columns = X.columns)
sample1

,Classes_per_week,Happy_with_instructors,Happy_with_class_duration,Happy_with_class_timings,Happy_with_class_size,Happy_with_facilities,Happy_with_price
0,3,Yes,Yes,Yes,Yes,No,No


In [81]:
pipe.predict(sample1)

array(['No'], dtype=object)

Add some more data to predict a 'Yes'

In [86]:
sample2 = [2,'No', 'Yes', 'No', 'No', 'Yes', 'No']
sample2 = pd.DataFrame([sample2], columns = X.columns)
sample2

,Classes_per_week,Happy_with_instructors,Happy_with_class_duration,Happy_with_class_timings,Happy_with_class_size,Happy_with_facilities,Happy_with_price
0,2,No,Yes,No,No,Yes,No


In [80]:
pipe.predict(sample2)

array(['Yes'], dtype=object)

Source: https://www.youtube.com/watch?v=irHhDMbw3xo